In [ ]:
import tensorflow as tf, keras
from keras import Model, layers, optimizers, losses
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import os, sys,concurrent.futures as futures
from PIL import Image

In [ ]:
channels = 3
conv_args = {
    "activation": "relu",
    "kernel_initializer": "Orthogonal",
    "padding": "same",
}
upscale_factor = 2
working_dir = 'o:/temp/pixiv/training/ds'

In [ ]:
il = layers.Input(shape=(None, None, channels))
x = layers.Conv2D(64, 3, **conv_args)(il)
x = layers.Conv2D(64, 3, **conv_args)(x)
x = layers.Conv2D(128, 3, **conv_args)(x)
x = layers.Conv2D(channels * (upscale_factor**2), 3, **conv_args)(x)
ol = tf.nn.depth_to_space(x, upscale_factor)
model = Model(il, ol)
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=losses.MeanSquaredError())


In [ ]:
def proccess_img(path):
    img = Image.open(path)
    dimg = img.resize(
        (img.width // upscale_factor, img.height // upscale_factor),
        Image.BICUBIC)
    x = (np.array(dimg) / 255.0).astype(np.float32)
    y = (np.array(img) / 255.0).astype(np.float32)
    return x, y


with futures.ThreadPoolExecutor(max_workers=6) as executor:
    img_paths = []
    for f in os.listdir(working_dir):
        if not (f.endswith('.png') or f.endswith('.jpg')):
            continue
        img_path = os.path.join(working_dir, f)
        img_paths.append(img_path)
    ds = list(executor.map(proccess_img, img_paths))
    dx = np.array([i[0] for i in ds])
    dy = np.array([i[1] for i in ds])


In [ ]:
esc = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    verbose=1,
    mode='min',
    restore_best_weights=True,
)
model.fit(dx,
          dy,
          batch_size=50,
          validation_split=0.1,
          epochs=1000,
          callbacks=[esc])


In [ ]:
img = Image.open('./test.png')
img_up = img.resize((img.width * upscale_factor, img.height * upscale_factor))
img_up.save('./test_up.png', quality=100)
img = (np.array(img) / 255.0).astype(np.float32)
img = np.expand_dims(img, axis=0)
res = model.predict(img)
print(res.shape)
keras.preprocessing.image.array_to_img(res[0]).save('./test_sr.png',
                                                    quality=100)

In [ ]:

model.save('./sr-espc.h5')

In [ ]:
model.summary()